# About: VCノードの作成

---

Moodle環境を構築するための仮想マシンをVCP SDKを用いて作成します。

## パラメータの設定

VCノードを作成するために必要となるパラメータを指定します。

### VCP SDK

VCP SDKを利用する際に必要となるパラメータを設定します。

![VCP SDK](images/moodle-011-02.png)

#### VCCアクセストークンの入力

VCノードを起動するにはVC Controller(VCC)にアクセスして、操作を行う必要があります。VCCにアクセスするために必要となるアクセストークンをここで入力します。

次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
config vc failed: http_status(403)
2020/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

> `unfreeze`するにはNotebookのツールバーにあるボタン<i class="fa-unfreeze fa"></i>、<i class="fa-unfreeze-below-in-section fa"></i>、<i class="fa-unfreeze-below-all fa"></i> を利用してください。

#### UnitGroup名の指定

このアプリケーションテンプレートで構築するMoodle環境に対して名前を付けます。指定した名前はVCPのUnitGroup名となります。

VCPの構成要素は以下のようになっています。

* VCノード
  - クラウドにおける計算資源(VM/BM)
  - 例えば Amazon EC2インスタンス, Microsoft Azure VM など
* Unit
  - 同質のVCノードにより構成されている要素
  - 同じUnitに属するVCノードはCPU,メモリ等の計算資源が全て同じ設定になっている
* UnitGroup
  - 複数のUnitにより構成されている要素
  - 使用目的、ライフサイクルなどに合わせて、複数のUnitをまとめて扱うための要素  

VCP SDKで作成した他の環境と区別するために UnitGroupに名前を付けます。UnitGroup名は既存のものと異なる値を指定する必要があります。
既存のUnitGroupを確認するために現在のUnitGroupの一覧を表示します。

In [ ]:
vcp.df_ugroups()

ここで構築する環境のUnitGroup名を次のセルで指定してください。

> UnitGroup名はアルファベットで開始され、その後に英数字またはアンダースコア`_`からなる文字列を指定してください。

In [ ]:
# (例)
# ugroup_name = 'Moodle'

ugroup_name =

#### パラメータの保存

この章で指定したパラメータの値をファイルに保存します。

後の手順でVCノードに対する操作を、構成管理ツールの[Ansible](https://www.ansible.com/)で行います。そこで、パラメータの保存形式は `Ansible` のフォーマットに従うことにします。Ansible では `group_vars/`というディレクトリに YAML フォーマットのファイルを配置すると、そのファイルに記録されている値を変数として利用することができます。このNotebookでは `group_vars/` にあるファイルを `group_vars ファイル`と呼ぶことにします。

値の保存を行う前に、入力されたパラメータに対して簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

In [ ]:
%run scripts/utils.py

check_parameters(
    'ugroup_name',
    params={'vcp': vcp},
    nb_vars=locals(),
)

次のセルを実行すると、この章で指定したパラメータが group_vars ファイルに保存されます。

In [ ]:
%run scripts/group.py
from pathlib import Path

update_group_vars(
    ugroup_name,
    ugroup_name=ugroup_name,
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{ugroup_name}.yml

### VCノード

Moodle環境を構築するVCノード（仮想マシン）に関するパラメータを指定します。

![VCノード設定](images/moodle-011-03.png)

#### クラウドプロバイダの指定

VCノードを起動するプロバイダを選択します。

In [ ]:
# (例)
# vc_provider = 'aws'
# vc_provider = 'azure'

vc_provider =

#### VCノードに割り当てるリソース量の指定


VCノードに割り当てるリソース量を`spec`オブジェクトに指定します。

VCノードの`spec`オブジェクトに対して全てのパラメータを毎回設定するのは煩雑です。そのためVCP SDKでは典型的な構成のパラメータセットを事前に定義してあります。事前に定義した`spec`パラメータセットのことを`flavor`と呼び、`spec`を作成する際に指定することになっています。

`spec`に設定できるパラメータはクラウドプロバイダ毎に異なるので 各`flavor`に対応するパラメータの値もプロバイダ毎に定義されています。
次のセルを実行すると `vc_provider` に設定したプロバイダに対応する `flavor` の一覧が表示されます。

In [ ]:
vcp.df_flavors(vc_provider)

表示された `flavor` の値から一つを選択して、次のセルに指定してください。

In [ ]:
# (例)
# vc_flavor = 'small'

vc_flavor =

`flavor`に設定されている以外のインスタンスタイプ(VMタイプ)を指定したい場合は、次のセルで`vc_vm_type`の値を指定してください。

> AWS, Azure の場合の設定例をコメントに記しています。必要に応じてコメントを外して設定を行ってください。

In [ ]:
# (例)
# vc_vm_type = 'm5.large'          # AWS
# vc_vm_type = 'Standard_D4_v3'    # Azure

#### ボリュームサイズの指定

In [ ]:
# (例)
# vc_disk_size = 32

vc_disk_size =

#### IPアドレスの指定

Moodleを実行するVCノードはHTTPサーバとなるので、事前に定めたIPアドレスを割り当てたほうが構築しやすくなります。
ここではVCノードのIPアドレスを指定します。

まず指定可能なIPアドレスの範囲を確認します。次のセルを実行するとVCノードに割り当て可能なIPアドレスの範囲が表示されます。

In [ ]:
print(vcp.get_vpn_catalog(vc_provider).get('private_network_ipmask'))

次のセルでVCノードに割り当てる （プライベート）IPアドレスを指定してください。

In [ ]:
# (例)
# vc_moodle_ipaddress = '172.30.2.100'

vc_moodle_ipaddress =

#### SSH公開鍵認証の鍵ファイルの指定

起動したVCノードにsshでログインして操作を行うために、事前にSSH公開鍵認証の公開鍵を登録する必要があります。

VCノードに登録するSSHの公開鍵ファイルのパスを次のセルで指定してください。

In [ ]:
# (例)
# ssh_public_key_path = '~/.ssh/id_rsa.pub'

ssh_public_key_path =

公開鍵に対応する秘密鍵のパスを次のセルで指定してください。

In [ ]:
# (例)
# ssh_private_key_path = '~/.ssh/id_rsa'

ssh_private_key_path =

#### パラメータの保存

この章で指定したパラメータの値をファイルに保存します。

値の保存を行う前に、入力されたパラメータに対して簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

In [ ]:
%run scripts/utils.py

check_parameters(
    'vc_provider',
    'vc_flavor',
    'vc_moodle_ipaddress',
    'ssh_public_key_path',
    'ssh_private_key_path',
    params={'vcp': vcp},
    nb_vars=locals(),
)

次のセルを実行すると、この章で指定したパラメータが group_vars ファイルに保存されます。

In [ ]:
%run scripts/group.py

update_group_vars(
    ugroup_name,
    vc_provider=vc_provider,
    vc_flavor=vc_flavor,
    vc_moodle_ipaddress=vc_moodle_ipaddress,
    ssh_public_key_path=str(ssh_public_key_path),
    ssh_private_key_path=str(ssh_private_key_path),
)
if 'vc_vm_type' in vars():
    update_group_vars(ugroup_name, vc_vm_type=vc_vm_type)
if 'vc_disk_size' in vars():
    update_group_vars(ugroup_name, vc_disk_size=vc_disk_size)

ここまでに保存したパラメータを確認するために group_vars ファイルの内容を表示します。

In [ ]:
!cat group_vars/{ugroup_name}.yml

## VCノードの作成

VCP SDK を利用してVCノードを作成します。

![VCノードの作成](images/moodle-011-04.png)

### VCノードの spec を指定する

前節で指定したパラメータをVCノードの `spec` に設定します。

`spec`に設定するパラメータを以下に示します。

* `images`: Baseコンテナイメージ
  - Baseコンテナイメージを設定します
* `params_v`: ボリューム設定
  - Baseコンテナのボリュームを設定します
  - Moodle用Baseコンテナではホスト側の `/sys/fs/cgroup`, `/lib/modules` をコンテナから見えるように設定する必要があります
* `ip_addresses`: IPアドレス
  - VCノードに割り当てるプライベートIPアドレスを設定します
* `set_ssh_publickey()`: SSHの公開鍵
  - VCノードに登録するSSHの公開鍵

In [ ]:
import os

gvars = load_group_vars(ugroup_name)
vc_provider = gvars['vc_provider']
spec = vcp.get_spec(vc_provider, gvars['vc_flavor'])

# Baseコンテナイメージを指定する
spec.image = 'harbor.vcloud.nii.ac.jp/vcp/moodle:1.8.1-base'
spec.params_v = [
    '/sys/fs/cgroup:/sys/fs/cgroup:ro',
    '/lib/modules:/lib/modules:ro',
]

# VCノードに割り当てるIPアドレスを指定する
spec.ip_addresses = [gvars['vc_moodle_ipaddress']]

# VCノードにsshでログインするための公開鍵を指定する
spec.set_ssh_pubkey(os.path.expanduser(gvars['ssh_public_key_path']))

# インスタンスタイプの指定
if 'vc_vm_type' in gvars:
    if vc_provider == 'aws':
        spec.instance_type = gvars['vc_vm_type']
    elif vc_provider == 'azure':
        spec.vm_size = gvars['vc_vm_type']
    
# ディスクサイズの指定
if 'vc_disk_size' in gvars:
    if vc_provider == 'aws':
        spec.volume_size = gvars['vc_disk_size']
    elif vc_provider == 'azure':
        spec.disk_size_gb = gvars['vc_disk_size']
        
# timezoneの指定
spec.params_e.append('TZ=Asia/Tokyo')

`spec` の設定値を確認します。

In [ ]:
print(spec)

### VCノードの起動

UnitGroupを作成します。

In [ ]:
ugroup = vcp.create_ugroup(ugroup_name)

VCノードを起動します。

In [ ]:
unit = ugroup.create_unit('moodle', spec)

VCノードの状態が `RUNNING` になっていることを確認します。

> VCノードの起動に失敗して`RUNNING`以外の状態になっている場合は次のセルを実行するとエラーになります。エラーになった場合は、`ugroup.cleanup()` を実行して VCノードを削除してください。

In [ ]:
if any([node.state != 'RUNNING' for node in unit.find_nodes()]):
    raise RuntimeError('ERROR: not running')

起動したVCノードの一覧を表示します。

In [ ]:
ugroup.df_nodes()

## Ansibleの設定

VCノードをAnsibleで操作するための設定を行います。

![Ansibleの設定](images/moodle-011-05.png)

まず、VCノードにSSHでログインできるようにするために `~/.ssh/known_hosts` の更新を行います。

> 何度かVCノードの起動を行うと、異なるホストが同じIPアドレスで起動するためにSSHのホストキーのチェックでエラーになる事があります。このような状況に対応するために、起動したVCノードのIPアドレスに対応するエントリを`known_hosts`ファイルから削除します。その後、`ssh-keyscan`コマンドを利用して起動したVCノードのホストキーを取得して `known_hosts`ファイルの内容を更新します。

In [ ]:
from time import sleep

def check_update_known_hosts(ipaddr):
    # VCノード起動直後だと sshd サービスが開始されておらずに known_hosts が更新されない場合がある
    # ssh-keyscan が値を取得できるまで何度かリトライする
    for x in range(10):
        out = ! echo $(ssh-keyscan {ipaddr} 2> /dev/null | wc -l)
        update_lines = int(out[0])
        if update_lines > 0:
            break
        sleep(1)
    else:
        raise RuntimeError("ERROR: timeout!")    

!mkdir -p -m 0700 ~/.ssh
!touch ~/.ssh/known_hosts
for addr in ugroup.find_ip_addresses():
    !ssh-keygen -R {addr}
    check_update_known_hosts(addr)
    !ssh-keyscan -H {addr} >> ~/.ssh/known_hosts
!mkdir -p ~/.ansible
!mkdir -p -m 0700 ~/.ansible/cp

起動したVCノードに対応するエントリを Ansible のインベントリに登録します。

> Ansibleで操作を行うためには、操作対象のホスト(IPアドレス)をインベントリに登録する必要があります。

In [ ]:
%run scripts/group.py
%run scripts/edit_conf.py

inventory = {'all': {'children': {
    ugroup.name: {
        'hosts': dict([(ip, {}) for ip in ugroup.find_ip_addresses()]),
        'vars': {
            'ansible_user': 'vcp',
            'ansible_ssh_private_key_file': gvars['ssh_private_key_path'],
            'ansible_python_interpreter': '/usr/bin/python',
        }
    }
}}}

generate_edit_link(update_inventory_yml(inventory))

次のセルを実行すると作成したインベントリの内容を表示します。インベントリの内容を変更したい場合は、上のセルの出力結果に表示しているリンクから編集することができます。

In [ ]:
!cat inventory.yml

先程VCノードを登録したファイルをインベントリとして指定するためのAnsibleのコンフィギュレーションファイルを作成します。

> カレントディレクトリにコンフィギュレーションファイル(`ansible.cfg`)を作成すると、Ansibleを実行する際にその設定が適用されます。

In [ ]:
cfg = setup_ansible_cfg()
generate_edit_link(cfg)

次のセルを実行すると作成したコンフィギュレーションファイルの内容を表示します。コンフィギュレーションファイルの内容を変更したい場合は、上のセルの出力結果に表示しているリンクから編集することができます。

In [ ]:
!cat ansible.cfg

VCノードに対して Ansible で接続できることを確認します。

In [ ]:
!ansible {ugroup_name} -m ping

正常に接続できると以下のように表示されます。

```
XXX.XXX.XXX.XXX | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
```

VCノードに対して設定ファイルの変更やパッケージの追加を行う場合にVCノードの管理者権限が必要になる場合があります。Ansibleで管理者権限によるコマンド実行が可能かどうかを確認します。

In [ ]:
# 管理者権限(-b)でのコマンド実行
!ansible {ugroup_name} -b -a 'whoami'